In [1]:
# Imports

import numpy as np
import os
from shapely.geometry import box
import pickle
import requests
import random
import shutil
import folium

In [2]:
# Configuration

# Set root DIR
root = "/Users/Kevin/Desktop"

# URL dummy for image request to openNRW server
WMS_1 = 'https://www.wms.nrw.de/geobasis/wms_nw_dop?SERVICE=WMS&REQUEST=GetMap&Version=1.1.1&LAYERS=nw_dop_rgb&SRS=EPSG:4326&BBOX='
WMS_2 = '&WIDTH=320&HEIGHT=320&FORMAT=image/png;%20mode=8bit'

# Avg. earth radius in meters
r = 6371000

# Square side length in meters
side = 16

# Distance in north-south direction
dlat = (side * 360) / (2 * np.pi * r)

In [3]:
Geos = {
    
       "Urban":{"Cologne": [50.937527,6.979710], "Düsseldorf": [51.223189, 6.780295], "Essen": [51.459341, 7.011598], 
                "Münster": [51.963765, 7.626303], "Dortmund": [51.512327, 7.466267], "Gelsenkirchen": [51.514505, 7.091419], 
                "Wuppertal": [51.258259, 7.144611], "Bergisch Gladbach":[50.992891, 7.134583], "Bottrop": [51.526934, 6.930604], 
                "Krefeld": [51.332267, 6.533540], "Gladbach": [51.191971, 6.438457], "Bonn": [50.731830, 7.100342], 
                "Bielefeld": [52.021317, 8.527046]}, 

       "Rural":{"Steinfurt": [52.119819, 7.393303], "Billerbeck": [51.979561, 7.292810], "Altenbeken": [51.763250, 8.947695], 
                "Warstein": [51.450236, 8.354575], "Lennestadt": [51.105435, 8.072227], "Breckerfeld": [51.259785, 7.467779], 
                "Langenberg": [51.360001, 7.125773], "Erkelenz": [51.082034, 6.314286], "Düren": [50.801748, 6.483726], 
                "Attendorn":[51.128264, 7.903772], "Gütersloh": [51.906335, 8.376942], "Lippstadt": [51.674822, 8.345715],
               "Borken": [51.843978, 6.858339]},
    
       "Openland":{"Nümbrecht":[50.874047, 7.540769], "Zülpich":[50.645783, 6.594761], "Alpen":[51.614790, 6.499519], 
                   "Warendorf": [51.883914, 7.934803], "Hövelhof": [51.836601, 8.578631], "Lichtenau": [51.652166, 8.967448],
                   "Menden": [51.396786, 7.892167], "Moers": [51.393177, 6.584675], "Haltern am See": [51.789994, 7.257793], 
                   "Schöppingen":[52.106305, 7.147231]},
    
       "Woods":{"Rothaarkamm": [51.085078, 8.350841], "Arnsberg": [51.439845, 7.893270]},

       "Water":{"Dhünntalsperre": [51.071188, 7.217944], "Möhnesee": [51.478785, 8.072667]},
    
       "Mountain":{"Kahler Asten": [51.180198, 8.488161]},
    
       "Mining":{"Etzweiler":[50.909085, 6.522354]},
    
        }

In [4]:
# Create a map using 
m = folium.Map(location=[50.5558095,9.6808449], zoom_start=6)

for category in Geos:
    
    for name, coords in Geos[category].items():
    
        folium.Marker(location=[coords[0],coords[1]]).add_to(m)

# Display map
m

In [ ]:
def download(lon, lat):

    y_max = lat + dlat/2
    x_min = lon - (((side * 360) / (2 * np.pi * r * np.cos(np.deg2rad(y_max))))/2)
    y_min = lat - dlat/2
    x_max = lon + (((side * 360) / (2 * np.pi * r * np.cos(np.deg2rad(y_max))))/2)
    # box(x_min, y_min, x_max, y_max) #fakultativ nur für plotting z.B. QGIS
    
    # Specify URL from which we download our tile
    url = os.path.join(WMS_1 + str(x_min) + ',' + str(y_min) + ',' + str(x_max) + ',' + str(y_max) + WMS_2)

    # Download tile imagery from URL
    response = requests.get(url, stream=True)
    
    # Save downloaded file under current_save_path
    with open(os.path.join(root,"openNRW_DSfG", str(lat) + ',' + str(lon) + ".png"), 'wb') as out_file:

        response.raw.decode_content = True
        shutil.copyfileobj(response.raw, out_file)

    del response

In [ ]:
for category in Geos:
    
    if category == "Urban":
        
        span = 0.02
        
    else:
        
        span = 0.01
    
    for name, coords in Geos[category].items():

        print(name, coords, span)
        
        # center point coordinates of your chosen location
        lat_center = coords[0]
        lon_center = coords[1]
        
        # create bounding box over chosen location
        rec_south = lat_center - (span/2)
        rec_north = lat_center + (span/2)
        rec_east = lon_center + (span/2)
        rec_west = lon_center - (span/2)
        
        for i in range(0, 850):
        
            lat = rec_south + (rec_north - rec_south) * random.random()
            lon = rec_west + (rec_east - rec_west) * random.random()
            
            download(lon, lat)  